<a href="https://colab.research.google.com/github/cesarmartinezg-lgtm/Entregas/blob/main/02%20-%20preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importación de Librerías

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt

# Configuración Kaggle

In [ ]:
# Establecer tus credenciales de Kaggle
os.environ['KAGGLE_USERNAME'] = 'cesarmartinezia'
os.environ['KAGGLE_KEY'] = '3bd3c2a5994356c24295cd5c6d8bba59'

In [ ]:
# Crear manualmente el archivo kaggle.json a partir de esas variables
!mkdir -p ~/.kaggle
with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write('{"username":"%s","key":"%s"}' % (os.environ['KAGGLE_USERNAME'], os.environ['KAGGLE_KEY']))

In [ ]:
# Ajustar permisos
!chmod 600 /root/.kaggle/kaggle.json

# Descarga de Dataset y cargue




In [ ]:
# Descargar los datos de la competencia
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
df = pd.read_csv("train.csv")


#Eliminar columnas irrelevantes

In [ ]:
cols_to_drop = [col for col in df.columns if "id" in col.lower() or "ID" in col]
df = df.drop(columns=cols_to_drop, errors='ignore')
df.head()


,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,20212,ENFERMERIA,BOGOTÁ,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,20212,DERECHO,ATLANTICO,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,20212,PSICOLOGIA,ANTIOQUIA,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


#Rellenar valores nulos

In [ ]:
for col in df.select_dtypes(include=np.number).columns:
    df[col] = df[col].fillna(df[col].mean())

for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].fillna(df[col].mode()[0])

df.isnull().sum()


,0
PERIODO_ACADEMICO,0
E_PRGM_ACADEMICO,0
E_PRGM_DEPARTAMENTO,0
E_HORASSEMANATRABAJA,0
F_ESTRATOVIVIENDA,0
F_TIENEINTERNET,0
F_EDUCACIONPADRE,0
F_TIENELAVADORA,0
F_TIENEAUTOMOVIL,0
E_PRIVADO_LIBERTAD,0


#Codificar variables categóricas

In [ ]:
label_encoders = {}
for col in df.select_dtypes(include="object").columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

df.head()


,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,20212,300,4,3,2,1,11,1,1,0,0,1,1,5,2,0.322,0.208,0.310,0.267
1,20212,249,3,0,2,0,10,1,0,0,0,1,0,11,1,0.311,0.215,0.292,0.264
2,20203,819,4,4,2,1,8,1,0,0,0,0,1,8,1,0.297,0.214,0.305,0.264
3,20195,14,26,0,3,1,4,1,0,0,0,1,1,8,0,0.485,0.172,0.252,0.190
4,20212,907,1,2,2,1,6,1,1,0,0,1,1,6,3,0.316,0.232,0.285,0.294


#Escalar variables numéricas

In [ ]:
scaler = StandardScaler()
df[df.select_dtypes(include=np.number).columns] = scaler.fit_transform(df.select_dtypes(include=np.number))
df.head()


,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,1.294094,-0.397280,-0.607837,0.367195,0.414985,0.344211,1.478205,0.384856,1.389589,-0.007007,-0.884073,0.298706,0.344211,-0.499193,0.453727,0.437002,-0.556223,0.813978,0.060296
1,1.294094,-0.569200,-0.715376,-1.569713,0.414985,-2.905191,1.166246,0.384856,-0.719637,-0.007007,-0.884073,0.298706,-2.905191,1.350522,-0.439030,0.346934,-0.481341,0.508180,0.016142
2,0.439801,1.352266,-0.607837,1.012831,0.414985,0.344211,0.542328,0.384856,-0.719637,-0.007007,-0.884073,-3.347774,0.344211,0.425665,-0.439030,0.232301,-0.492038,0.729034,0.016142
3,-0.319570,-1.361384,1.758009,-1.569713,1.298980,0.344211,-0.705508,0.384856,-0.719637,-0.007007,-0.884073,0.298706,0.344211,0.425665,-1.331788,1.771650,-0.941332,-0.171371,-1.072993
4,1.294094,1.648914,-0.930452,-0.278441,0.414985,0.344211,-0.081590,0.384856,1.389589,-0.007007,-0.884073,0.298706,0.344211,-0.190907,1.346484,0.387874,-0.299484,0.389259,0.457683


#Guardar el dataset preprocesado

In [ ]:
df.to_csv("train_preprocesado.csv", index=False)
df.head()


,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,1.294094,-0.397280,-0.607837,0.367195,0.414985,0.344211,1.478205,0.384856,1.389589,-0.007007,-0.884073,0.298706,0.344211,-0.499193,0.453727,0.437002,-0.556223,0.813978,0.060296
1,1.294094,-0.569200,-0.715376,-1.569713,0.414985,-2.905191,1.166246,0.384856,-0.719637,-0.007007,-0.884073,0.298706,-2.905191,1.350522,-0.439030,0.346934,-0.481341,0.508180,0.016142
2,0.439801,1.352266,-0.607837,1.012831,0.414985,0.344211,0.542328,0.384856,-0.719637,-0.007007,-0.884073,-3.347774,0.344211,0.425665,-0.439030,0.232301,-0.492038,0.729034,0.016142
3,-0.319570,-1.361384,1.758009,-1.569713,1.298980,0.344211,-0.705508,0.384856,-0.719637,-0.007007,-0.884073,0.298706,0.344211,0.425665,-1.331788,1.771650,-0.941332,-0.171371,-1.072993
4,1.294094,1.648914,-0.930452,-0.278441,0.414985,0.344211,-0.081590,0.384856,1.389589,-0.007007,-0.884073,0.298706,0.344211,-0.190907,1.346484,0.387874,-0.299484,0.389259,0.457683
